In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

import lightning as L
from torch.utils.data import TensorDataset, DataLoader

In [3]:
class LSTMbyHand(L.LightningModule):
  def __init__(self):
    super().__init__()
    mean =torch.tensor(0.0)
    std = torch.tensor(1.0)

    self.wlr1 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
    self.wlr2 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
    self.blr1 = nn.Parameter(torch.tensor(0.), requires_grad=True)

    self.wpr1 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
    self.wpr2 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
    self.bpr1 = nn.Parameter(torch.tensor(0.), requires_grad=True)

    self.wp1 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
    self.wp2 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
    self.bp1 = nn.Parameter(torch.tensor(0.), requires_grad=True)

    self.wo1 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
    self.wo2 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
    self.bo1 = nn.Parameter(torch.tensor(0.), requires_grad=True)

  def lstm_unit(self, input_value, long_memory, short_memory):
    long_remember_percent = torch.sigmoid((short_memory * self.wlr1 + input_value * self.wlr2) + self.blr1)

    potential_remember_percent = torch.sigmoid((short_memory * self.wpr1 + input_value * self.wpr2) + self.bpr1)
    potential_memory = torch.tanh((short_memory * self.wp1 + input_value * self.wp2) + self.bp1)

    updated_long_memory = long_memory * long_remember_percent + potential_memory * potential_remember_percent

    output_percent = torch.sigmoid((short_memory * self.wo1 + input_value * self.wo2) + self.bo1)
    updated_short_memory = torch.tanh(updated_long_memory) * output_percent
    return ([updated_long_memory, updated_short_memory])
  
  def forward(self, input):
    long_memory = 0
    short_memory = 0

    day1 = input[0]
    day2 = input[1]
    day3 = input[2]
    day4 = input[3]

    long_memory, short_memory = self.lstm_unit(day1, long_memory, short_memory)
    long_memory, short_memory = self.lstm_unit(day2, long_memory, short_memory)
    long_memory, short_memory = self.lstm_unit(day3, long_memory, short_memory)
    long_memory, short_memory = self.lstm_unit(day4, long_memory, short_memory)

    return short_memory
  
  def configure_optimizers(self):
    return Adam(self.parameters())
  
  def training_step(self, batch, batch_idx):
    input_i, label_i = batch
    output_i = self.forward(input_i[0])
    loss = (output_i - label_i)**2

    self.log("train_loss", loss)

    if (label_i == 0):
      self.log("out_0", output_i)
    else:
      self.log("out_1", output_i)

    return loss

In [4]:
model = LSTMbyHand()

In [7]:
inputs = torch.tensor([[0., 0.5, 0.25, 1.], [1., 0.5, 0.25, 1.]])
labels = torch.tensor([0., 1.])

dataset = TensorDataset(inputs, labels)
dataloader= DataLoader(dataset)

In [8]:
trainer = L.Trainer(max_epochs = 2000)
trainer.fit(model, train_dataloaders=dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: c:\Users\igor\Documents\GitHub\financial_tools\currency\pytorch\lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type | Params
--------------------------------------
  | other params | n/a  | 12    
--------------------------------------
12        Trainable params
0         Non-trainable params
12        Total params
0.000     Total estimated model params size (MB)
c:\Users\igor\AppData\Local\Programs\Python\Python312\Lib\site

Epoch 1999: 100%|██████████| 2/2 [00:00<00:00, 153.86it/s, v_num=0]

`Trainer.fit` stopped: `max_epochs=2000` reached.


Epoch 1999: 100%|██████████| 2/2 [00:00<00:00, 90.91it/s, v_num=0] 


In [9]:
print(model(torch.tensor([0., 0.5, 0.25, 1.]).detach()))
print(model(torch.tensor([1., 0.5, 0.25, 1.]).detach()))

tensor(7.0823e-05, grad_fn=<MulBackward0>)
tensor(0.9174, grad_fn=<MulBackward0>)


In [10]:
path_to_best_checkpoint = trainer.checkpoint_callback.best_model_path

In [11]:
trainer = L.Trainer(max_epochs = 3000)
trainer.fit(model, train_dataloaders=dataloader, ckpt_path = path_to_best_checkpoint)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at c:\Users\igor\Documents\GitHub\financial_tools\currency\pytorch\lightning_logs\version_0\checkpoints\epoch=1999-step=4000.ckpt
c:\Users\igor\AppData\Local\Programs\Python\Python312\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:361: The dirpath has changed from 'c:\\Users\\igor\\Documents\\GitHub\\financial_tools\\currency\\pytorch\\lightning_logs\\version_0\\checkpoints' to 'c:\\Users\\igor\\Documents\\GitHub\\financial_tools\\currency\\pytorch\\lightning_logs\\version_1\\checkpoints', therefore `best_model_score`, `kth_best_model_path`, `kth_value`, `last_model_path` and `best_k_models` won't be reloaded. Only `best_model_path` will be reloaded.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type | Params
--------------------------------------
  | other par

Epoch 2999: 100%|██████████| 2/2 [00:00<00:00, 142.85it/s, v_num=1]

`Trainer.fit` stopped: `max_epochs=3000` reached.


Epoch 2999: 100%|██████████| 2/2 [00:00<00:00, 100.01it/s, v_num=1]


In [12]:
print(model(torch.tensor([0., 0.5, 0.25, 1.]).detach()))
print(model(torch.tensor([1., 0.5, 0.25, 1.]).detach()))

tensor(1.5967e-05, grad_fn=<MulBackward0>)
tensor(0.9566, grad_fn=<MulBackward0>)


In [13]:
path_to_best_checkpoint = trainer.checkpoint_callback.best_model_path

In [14]:
trainer = L.Trainer(max_epochs = 5000)
trainer.fit(model, train_dataloaders=dataloader, ckpt_path = path_to_best_checkpoint)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at c:\Users\igor\Documents\GitHub\financial_tools\currency\pytorch\lightning_logs\version_1\checkpoints\epoch=2999-step=6000.ckpt
c:\Users\igor\AppData\Local\Programs\Python\Python312\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:361: The dirpath has changed from 'c:\\Users\\igor\\Documents\\GitHub\\financial_tools\\currency\\pytorch\\lightning_logs\\version_1\\checkpoints' to 'c:\\Users\\igor\\Documents\\GitHub\\financial_tools\\currency\\pytorch\\lightning_logs\\version_2\\checkpoints', therefore `best_model_score`, `kth_best_model_path`, `kth_value`, `last_model_path` and `best_k_models` won't be reloaded. Only `best_model_path` will be reloaded.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type | Params
--------------------------------------
  | other par

Epoch 4999: 100%|██████████| 2/2 [00:00<00:00, 133.34it/s, v_num=2]

`Trainer.fit` stopped: `max_epochs=5000` reached.


Epoch 4999: 100%|██████████| 2/2 [00:00<00:00, 100.00it/s, v_num=2]


In [15]:
class LightningLSTM(L.LightningModule):
  def __init__(self):
    super().__init__()

    self.lstm = nn.LSTM(input_size=1, hidden_size=1)

  def forward(self, input):
    input_trans = input.view(len(input), 1)

    lstm_out, temp = self.lstm(input_trans)

    prediction = lstm_out[-1]
    return prediction
  
  def configure_optimizers(self):
    return Adam(self.parameters(), lr= 0.1)
  
  def training_step(self, batch, batch_idx):
    input_i, label_i = batch
    output_i = self.forward(input_i[0])
    loss = (output_i - label_i)**2

    self.log("train_loss", loss)

    if (label_i == 0):
      self.log("out_0", output_i)
    else:
      self.log("out_1", output_i)

    return loss

In [16]:
model = LightningLSTM()

In [17]:
trainer = L.Trainer(max_epochs = 300, log_every_n_steps=2)
trainer.fit(model, train_dataloaders=dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type | Params
------------------------------
0 | lstm | LSTM | 16    
------------------------------
16        Trainable params
0         Non-trainable params
16        Total params
0.000     Total estimated model params size (MB)
c:\Users\igor\AppData\Local\Programs\Python\Python312\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Epoch 299: 100%|██████████| 2/2 [00:00<00:00, 190.09it/s, v_num=3]

`Trainer.fit` stopped: `max_epochs=300` reached.


Epoch 299: 100%|██████████| 2/2 [00:00<00:00, 133.14it/s, v_num=3]
